In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/huggingfacetoken/huggingface_token.txt


In [ ]:
# Install required libraries
!pip install -q -U keras-nlp
!pip install -q -U transformers datasets bitsandbytes peft evaluate sentence-transformers

import os
import torch
import keras_nlp
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from peft import get_peft_model, LoraConfig, TaskType
from accelerate import Accelerator
from sentence_transformers import SentenceTransformer, util

# Read the Hugging Face token from file
with open('/kaggle/input/huggingfacetoken/huggingface_token.txt', 'r') as file:
    hf_token = file.read().strip()

# Log in to Hugging Face
from huggingface_hub import login
login(token=hf_token)

# Disable parallelism in tokenizers
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# 1. Load Dataset
dataset = load_dataset("Amod/mental_health_counseling_conversations")

# 2. Preprocess Data
def preprocess_function(examples):
    for i in range(len(examples['Response'])):
        examples['Response'][i] = "<user> " + examples['Response'][i] + " <counselor> "
    return examples

dataset = dataset.map(preprocess_function, batched=True)

# Split dataset
train_test_split = dataset['train'].train_test_split(test_size=0.1, seed=42)
train_data = train_test_split['train']
val_data = train_test_split['test']

# 3. Model and Tokenizer Loading
model_name = "facebook/bart-large"  # Example model for Seq2Seq
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, token=hf_token)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 4. Add Trainable Adapters
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=['self_attn.q_proj', 'self_attn.k_proj', 'self_attn.v_proj'],
)
model = get_peft_model(model, peft_config)

# 5. Custom Dataset Class
class CustomDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            padding="max_length",
            return_tensors='pt',
        )
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        labels = input_ids.clone()
        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

# 6. Prepare Data
train_dataset = CustomDataset(train_data['Response'], tokenizer, max_len=64)  # Reduced max_len
val_dataset = CustomDataset(val_data['Response'], tokenizer, max_len=64)

# Initialize Accelerator
accelerator = Accelerator()
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)

# Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * 3)
model, optimizer, train_dataloader, val_dataloader = accelerator.prepare(model, optimizer, train_dataloader, val_dataloader)

# Training Loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

# Manual Evaluation Loop
model.eval()  # Set model to evaluation mode
eval_loss = 0.0

with torch.no_grad():
    for batch in val_dataloader:
        outputs = model(**batch)
        eval_loss += outputs.loss.item()

# Calculate average evaluation loss and perplexity
eval_loss /= len(val_dataloader)
perplexity = torch.exp(torch.tensor(eval_loss))
print(f"Evaluation Perplexity: {perplexity}")

# Save the fine-tuned model and tokenizer
model.save_pretrained("./gemma-finetuned-mental-health")
tokenizer.save_pretrained("./gemma-finetuned-mental-health")

# Load a sentence transformer model for semantic similarity
sentence_model = SentenceTransformer('all-mpnet-base-v2')

# Modified Chatbot Function (using Sentence Similarity)
def semantic_chatbot_response(user_input):
    max_input_length = tokenizer.model_max_length - 10  # Keep this for safety
    user_input = user_input[:max_input_length]

    # Encode the user input
    user_embedding = sentence_model.encode(user_input)

    # Find the most similar response in the training data
    highest_similarity = -1
    most_similar_response = ""

    for text in train_data['Response']:  # Search through the training data
        response_embedding = sentence_model.encode(text)
        similarity = util.cos_sim(user_embedding, response_embedding)[0][0].item()  # Cosine similarity
        if similarity > highest_similarity:
            highest_similarity = similarity
            most_similar_response = text

    # If the similarity is very low, generate a response using the model
    if highest_similarity < 0.3:  # Adjust threshold as needed
        print("(Generating response with language model...)")  # Indicate fallback
        input_ids = tokenizer(user_input, return_tensors="pt").input_ids.to(model.device)
        with torch.no_grad():
            output = model.generate(
                input_ids,
                max_new_tokens=50,  # Adjust max_new_tokens for natural responses
                do_sample=True,
                top_k=50,  # Adjust top_k for more diverse sampling
                temperature=0.7,  # Adjust temperature for more natural responses
                eos_token_id=tokenizer.eos_token_id  # Stop at the end of sentence
            )
        most_similar_response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Additional steps to make responses more concise:
    # 1. Split into sentences and take the first few:
    sentences = most_similar_response.split('. ')  # Split based on periods
    concise_response = ". ".join(sentences[:2])  # Take the first two sentences (adjust as needed)

    return concise_response

def run_chatbot():
    print("Welcome to the Mental Health Chatbot! Type 'exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Take care! I'm here if you need to talk again.")
            break
        response = semantic_chatbot_response(user_input)
        print(f"Chatbot: {response}")

# Start the chatbot
run_chatbot()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


README.md:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

combined_dataset.json:   0%|          | 0.00/4.79M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Chatbot: <user> A great place to start is at www.psychologytoday.com.  You can search by location and identify therapists who have expertise in your area of concern.  Once you find a few people that fit your criteria, read about them on Psychology today as well as check out their website if they have one.  Next, either call or email them to set up a time to talk for a few minutes.  When you speak with them, you want to know a few things: do they have a decent amount of experience in your area of concern?  Are their fees, insurance coverage and scheduling availability a good fit for you?  Most importantly, see if you feel comfortable when you speak to them on the phone!  If this person is a good match in these areas, schedule an initial consultation so that you can see if you are also a good match in person. <counselor> 
